In [35]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

import pandas as pd
df = pd.read_csv('loan_dataset.csv')
df = df.dropna()
cols = [ 'Gender', 'Married', 'Dependents', 'Education','Self_Employed', 'ApplicantIncome', 
'CoapplicantIncome', 'LoanAmount','Loan_Amount_Term', 'Credit_History', 'Property_Area']
Xs = df[cols]
Ys = df['Loan_Status']


In [36]:
from sklearn.model_selection import train_test_split    
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector as selector
from sklearn.ensemble import RandomForestClassifier

numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer()), ("scaler", StandardScaler())]
)

categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="object")),
        ("cat", categorical_transformer, selector(dtype_include="object")),
    ]
)
clf = Pipeline(
steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier())])

X_train, X_test, y_train, y_test = train_test_split(
    Xs, 
    Ys, 
    test_size=0.2, 
    random_state=42
    )


In [37]:
from sklearn.ensemble import RandomForestClassifier

reg = clf
reg.fit(X_train, y_train)
# now, make sure the model is available downstream
model = reg
# go to the testing phase

In [38]:
from sklearn import metrics
import numpy as np
pred = model.predict(X_test)
acc = metrics.accuracy_score(y_test, pred)
print('The  model has Accuracy : {}'.format(acc))        

The  model has Accuracy : 0.8333333333333334


In [39]:
per_test = X_test

In [62]:
per_test['test_Status'] = y_test
per_test['test_Status'] = per_test['test_Status'].replace({'Y':1, 'N':0})

In [65]:
per_test['pred_Status'] = pred
per_test['pred_Status']= per_test['pred_Status'].replace({'Y':1, 'N':0})
per_test.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,test_Status,pred_Status
92,Male,Yes,2,Not Graduate,No,3273,1820.0,81.0,360.0,1.0,Urban,1,1
529,Male,No,0,Not Graduate,No,6783,0.0,130.0,360.0,1.0,Semiurban,1,1
505,Male,Yes,2,Graduate,No,3510,4416.0,243.0,360.0,1.0,Rural,1,1
358,Male,Yes,0,Not Graduate,No,3000,1666.0,100.0,480.0,0.0,Urban,0,0
512,Male,Yes,2,Graduate,No,3283,2035.0,148.0,360.0,1.0,Urban,1,1


In [69]:
from sklearn.metrics import confusion_matrix
Male = per_test.loc[df['Gender'] == 'Male']
Female = per_test.loc[df['Gender'] == 'Female']
y_true = Male['test_Status']
y_pred = Male['pred_Status']
CM = confusion_matrix(y_true, y_pred)
CM



array([[10, 11],
       [ 0, 59]])